Table of Contents

1. Import Libraries

2. Import Data

3. Combine products dataframe with new df_ords_prods_combined dataframe

4. Data Consistency Checks

5. Combining df_ords_prods_combined with df_prods

6. Export Dataframe

1. Import Libraries

In [28]:
# Import Libraries
import pandas as pd
import numpy as np
import os

2. Import Data

In [29]:
# Create path
path = '/Users/ianfleming/Desktop/05-2025 Instacart Basket Analysis'

Step 3. Import Combined Dataframes

In [30]:
# Import combined data for products and orders
df_ords_prods_combined = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl' ))

In [31]:
# Check output
df_ords_prods_combined

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both
...,...,...,...,...,...,...,...,...,...,...,...
32434484,2977660,206209,13,1,12,7.0,False,14197,5,1,both
32434485,2977660,206209,13,1,12,7.0,False,38730,6,0,both
32434486,2977660,206209,13,1,12,7.0,False,31477,7,0,both
32434487,2977660,206209,13,1,12,7.0,False,6567,8,0,both


Step 4. Check Shape of Data

In [32]:
# Check Shape
df_ords_prods_combined.shape

(32434489, 11)

Step 5. Combine products dataframe with new df_ords_prods_combined dataframe

3. Combine products dataframe with new df_ords_prods_combined dataframe

In [33]:
# Import products dataframe
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'), index_col = False)

In [34]:
# Check import output
df_prods

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3
...,...,...,...,...,...
49667,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,5.3
49668,49685,En Croute Roast Hazelnut Cranberry,42,1,3.1
49669,49686,Artisan Baguette,112,3,7.8
49670,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,4.7


Since I am combining dataframes using the column 'product_id', I will double check for duplicates in this column, as my previous duplicate check only checks for duplicate rows, where data across each column is a duplicate. 

4. Data Consistency Checks

In [35]:
# Check for duplicates
df_prods['product_id'].duplicated().sum()

2

In [36]:
# To see the duplicates
df_prods[df_prods['product_id'].duplicated(keep=False)].sort_values('product_id')

,product_id,product_name,aisle_id,department_id,prices
6784,6800,Revive Zero Vitamin Water,64,7,6.4
6785,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26504,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26505,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


Two product ids are used for two different product items. This is a data quality issue. In a real world situation; I would flag this as a critical data integrity issue. For this situation though, I will just drop one of each product.

In [37]:
# Drop duplicates
df_prods = df_prods.drop_duplicates(subset='product_id')

In [38]:
# Test merge as to not overwrite
df_test_merge = df_ords_prods_combined.merge(df_prods, on='product_id', how='left')

In [39]:
df_test_merge.shape

(32434489, 15)

5. Combining df_ords_prods_combined with df_prods

In [40]:
# Combining df_ords_prods_combined with df_prods using column 'product_id' and a left join
df_ords_prods_merged = df_ords_prods_combined.merge(df_prods, on = 'product_id', how='left', indicator = 'match')

In [41]:
# Check merge output
df_ords_prods_merged

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,match
0,2539329,1,1,2,8,NaN,True,196,1,0,both,Soda,77.0,7.0,9.0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both,Original Beef Jerky,23.0,19.0,4.4,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both,Aged White Cheddar Popcorn,23.0,19.0,4.7,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,2977660,206209,13,1,12,7.0,False,14197,5,1,both,Tomato Paste,9.0,9.0,5.6,both
32434485,2977660,206209,13,1,12,7.0,False,38730,6,0,both,Brownie Crunch High Protein Bar,3.0,19.0,5.9,both
32434486,2977660,206209,13,1,12,7.0,False,31477,7,0,both,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,both
32434487,2977660,206209,13,1,12,7.0,False,6567,8,0,both,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,both


In [42]:
# Check output
df_ords_prods_merged['match'].value_counts()

match
both          32404289
left_only        30200
right_only           0
Name: count, dtype: int64

In [43]:
# Check shape
df_ords_prods_merged.shape

(32434489, 16)

Both dataframes are merged. I have preserved the 32434489 rows from the original dataframe 'df_ords_prods_combined' and merged 'df_prods' using a left join and the column 'product_id'. After merging, I used the 'indicator = True' argument to check the match results. The majority of the rows had matching 'product ids' in both dataframes, while 30,200 had product_id rows that only existed in the left dataframe (df_ords_prods_combined). 

But i now have 16 columns because of the two flag columns so I will keep the one most relevant 'match'

In [44]:
# Drop '_merge' column
df_ords_prods_merge = df_ords_prods_merged.drop(columns=['_merge'])

In [45]:
# Check shape
df_ords_prods_merge.shape

(32434489, 15)

In [46]:
# Test merge using an outerjoin
df_merge_check = pd.merge(df_ords_prods_combined, df_prods, on ='product_id', how='outer', indicator='merge_flag')

In [47]:
# Check output
df_merge_check['merge_flag'].value_counts()

merge_flag
both          32404289
left_only        30200
right_only          11
Name: count, dtype: int64

The 11 product_ids that appear in the right dataframe (df_prods) may be new products not yet purchased or old products nobdy bought or data errors. 

6. Export Dataframe

In [48]:
# Export data as .pkl
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))